In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utilsCM

from tqdm import tqdm

In [2]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../code-00-preprocessdataset/'
Y_embeddings = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
Y_embeddings = Y_embeddings.values[:,1:Y_embeddings.shape[1]-1].astype(np.float)

In [3]:
# keyword = {'ROIpred'}
keyword = {'DNNActvtn'}

# layer =  {'conv_1', 'conv_2','conv_3','conv_4','conv_5'}
layer =  {'conv_1'}

# ROI = {'EVC','LOC','PFS','OPA','PPA','RSC','FFA','OFA','STS','EBA'}
# ROI = {'EVC','LOC','PPA'}
ROI = {'EVC'}

Sub = [1,2,3,4]
# Sub = [1]

#how many components to keep?
Keepncomps = [20]
# Keepncomps = [10,40,80]
pretrained_val = True

datapath = '../../../data-00/'
savepath = '../../../data-01/'

In [4]:
for ilayer in layer:
    for ikeyword in keyword:
        
        if ikeyword is 'ROIpred':
            
            for iROI in ROI:
                
                predictor_variable = {}
                for iSub in Sub:
                    if not pretrained_val:
                        thisSub = np.load(datapath +  "ROIpred_Sub" + str(iSub) + '_' + iROI + "_" + ilayer + '_untrained.npy')
                    else:
                        thisSub = np.load(datapath +  "ROIpred_Sub" + str(iSub) + '_' + iROI + "_" + ilayer + '.npy')
                            
                    #load ROIpred as predictor variable
                    if iSub is 1:
                        predictor_variable = thisSub
                    else:
                        predictor_variable = np.append( predictor_variable , thisSub, axis = 1)

                for icomps in Keepncomps:
                    mean_r = {}
                    mean_r = utilsCM.iter_cvregress(predictor_variable,Y_embeddings,ikeyword,ilayer,icomps,iROI,savefolder = savepath, pretrained = pretrained_val)
#                     utilsCM.make_figure(mean_r,ikeyword,ilayer,icomps,iROI, figurepath = 'figures-01/')
#                     print(mean_r)
        
        
        elif ikeyword is 'DNNActvtn':            
            if not pretrained_val:
                predictor_variable = pd.read_csv(datapath +  "things_" + ilayer + '_untrained.csv', header=None, index_col=0).iloc[:,:].to_numpy()
            else:
                predictor_variable = pd.read_csv(datapath +  "things_" + ilayer + '.csv', header=None, index_col=0).iloc[:,:].to_numpy()
            
            for icomps in Keepncomps:
                
                mean_r = {}
                mean_r = utilsCM.iter_cvregress(predictor_variable,Y_embeddings,ikeyword,ilayer,icomps,savefolder = savepath, pretrained = pretrained_val)
#                 utilsCM.make_figure(mean_r,ikeyword,ilayer,icomps, figurepath = 'figures-01/')

k-fold regression, independet variable: 20 PCs retained of DNNActvtn from conv_1


/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/semantic-code/cm/code-01-analysis/utilsCM.py:172: RuntimeWarning: Mean of empty slice
  mean_r = np.nanmean(rs, axis=0) #TO handle Nans, since the feature space is so sparse


In [5]:
tresh_bonf = utilsCM.p2r(.05/Y_embeddings.shape[1], 1470)
figurepath='../../../figures-02/'

In [6]:
##Showing histograms for senses above threshold
myDict = {}

for ilayer in layer:
    
    for ikeyword in keyword:        
        
        for icomps in Keepncomps:
            thisPrediction = []
            
            if ikeyword is 'DNNActvtn':
                if not pretrained_val:
                    filename = 'PredictSENSES_' + ikeyword + '_'+ ilayer + '_'+ str(icomps) +'PCs_untrained'
                else:
                    filename = 'PredictSENSES_' + ikeyword + '_'+ ilayer + '_'+ str(icomps) +'PCs'
                thisPrediction = np.load('../../../data-01/' + filename + '.npy')
                pred_thresh = thisPrediction[thisPrediction>tresh_bonf]
                utilsCM.make_figure(pred_thresh,ikeyword,ilayer,icomps,figure_size=(50,20),figure_path = figurepath,font_size=40,pretrained = pretrained_val)
#                 myDict.update(pred_thresh.shape[0])
                if ikeyword not in myDict:
                    myDict[ikeyword] = []
                myDict[ikeyword].append(pred_thresh.shape[0])
            
            elif ikeyword is 'ROIpred':
                
                for iROI in ROI:
                    thisPrediction = []
                    if not pretrained_val:
                        filename = 'PredictSENSES_' + ikeyword + '_' +iROI + '_'+ ilayer + '_'+ str(icomps) +'PCs_untrained'
                    else:
                        filename = 'PredictSENSES_' + ikeyword + '_' +iROI + '_'+ ilayer + '_'+ str(icomps) +'PCs'
                    thisPrediction = np.load(savepath + filename + '.npy')
                    pred_thresh = thisPrediction[thisPrediction>tresh_bonf]
                    utilsCM.make_figure(pred_thresh,ikeyword,ilayer,icomps,iROI,figure_size=(50,20),figure_path = figurepath,font_size=40,pretrained = pretrained_val)
                    if iROI not in myDict:
                        myDict[iROI] = []
                    myDict[iROI].append(pred_thresh.shape[0])

In [7]:
filename_PPA = 'PredictSENSES_' + 'ROIpred' + '_'+ 'PPA'+'_'+ ilayer+ '_'+ str(icomps) +'PCs'
thisPrediction_PPA = np.load(savepath + filename_PPA + '.npy')

filename_EVC = 'PredictSENSES_' + 'ROIpred' + '_'+ 'EVC'+'_'+ ilayer + '_'+ str(icomps) +'PCs'
thisPrediction_EVC = np.load( savepath + filename_EVC + '.npy')

filename_LOC = 'PredictSENSES_' + 'ROIpred' + '_'+ 'LOC'+'_'+ ilayer + '_'+ str(icomps) +'PCs'
thisPrediction_LOC = np.load(savepath + filename_LOC + '.npy')

# filename_DNN = 'PredictSENSES_' + 'DNNActvtn' +'_'+ ilayer + '_'+ str(icomps) +'PCs'
# thisPrediction_DNN = np.load('../../../data-01/' + filename_DNN + '.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../../../data-01/PredictSENSES_ROIpred_PPA_conv_1_20PCs.npy'

In [ ]:
thisPrediction_PPA

In [ ]:
thisPrediction_EVC

In [ ]:
thisPrediction_LOC